In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree

import requests
from lxml import html


import re
import hashlib
import sqlite3
import telegram_send
import telebot
import time
import configparser

In [2]:
#Ограничивает длину слов
WORLD_LIMIT = 300

In [3]:


# Создание объекта конфигурации
config = configparser.ConfigParser()

# Чтение данных из файла
config.read('config.ini')

BOT_TOKEN = config.get('tokens', 'BOT_TOKEN')
CHAT_ID = config.get('tokens', 'CHAT_ID')

## Функции

In [4]:
def get_div(tree, xpath_format, div_mode=True):
    list_divs = []


    # Выбор элемента
    selected_element = tree.xpath(xpath_format)


    for div in selected_element:
        # Получение HTML-разметки только для выбранного элемента
        html_representation = html.tostring(div, encoding='unicode', method='html')

        list_divs.append(html_representation)
    return list_divs


# Parse

In [5]:
dict_info = {}



d_mus_names = \
{
# 'https://victorymuseum.ru/playbill/' :'Музей победы',
# 'https://blokadamus.ru/музей-обороны-и-блокады-ленинграда/события/':'Музей блокады Ленинграда',
# 'http://www.cmaf.ru/':'Музей минобороны в Москве',
# 'https://1418museum.ru/news/':'Музей дорогами победы (парк Патриот)',
# 'https://1812panorama.ru/afisha':'Бородинская битва',
# 'https://sovrhistory.ru/exhibitions/future/index.php':'Музей современной истории России',
# 'https://moscowmanege.ru/ru/vystavki-all/':'Выставочный центр Манеж, Москва',
# 'https://mosmuseum.ru/exhibitions/':'Музей Москвы',
# 'https://mediashm.ru/?all=1' :'ГИМ, Москва',
# 'https://www.museikino.ru/media_center/':'Музей кино, Москва',
# 'https://ugvim.ru/news/':"Музей военной истории «Свердловск: Говорит Москва!» ",
# 'https://www.leninmemorial.ru/news/':'Дом-музей В.И. Ленина',
# 'http://мемориал.бгикм.рф/меню/новости':'Мемориал «В честь героев Курской битвы»',
# 'http://patriotrm.clan.su/news/?page1':'Мемориальный музей военного и трудового подвига 1941–1945 гг',
# 'https://прохоровское-поле.рф/vystavochnaya-deyatelnost.html':'ГОСУДАРСТВЕННЫЙ ВОЕННО-ИСТОРИЧЕСКИЙ МУЗЕЙ-ЗАПОВЕДНИК «ПРОХОРОВСКОЕ ПОЛЕ»',    
# 'https://museum-t-34.ru/новости/':'Музейно-мемориальный комплекс «История танка Т-34»',
# 'https://derbentmuseum.ru/':'Музей «Боевая слава» г. Дербента',
# 'https://memorial24.ru/novosti':'Музей «Мемориал Победы»',
# 'https://www.smolensk-museum.ru/novosti/muzeynaya_zhizn/':'Музей «Смоленщина в годы Великой Отечественной войны 1941−1945 гг.',
# 'https://www.lenoblmus.ru/news':'Музей «Невский пятачок',
# 'https://navalmuseum.ru/news':'Музей «Дорога жизни»',
# 'https://alabin.ru/':'Самарский областной историко-краеведческий музей им. П.В. Алабина',
# 'https://yakutmuseum.ru/':'Якутский государственный объединенный музей истории и культуры народов Севера им. Ем. Ярославского',
# 'https://felicina.ru/news/':'Краснодарский государственный историко-археологический музей-заповедник им. Е.Д. Фелицына',
# 'http://www.chelmuseum.ru/news/#content':'Государственный исторический музей Южного Урала',
# 'https://tmbmuseum.ru/mainmenu/news/relevant':'Тамбовский областной краеведческий музей',
# 'https://museumrb.ru/category/afisha/events/':'https://www.museumrb.ru/o-muzee/',
# 'https://stavmuseum.ru/':'Ставропольский государственный музей-заповедник',
# 'https://youmuseum.ru/news/':'Новосибирский государственный краеведческий музей',
# 'http://ogikm.ru/':'Оренбургский губернаторский историко-краеведческий музей ',
'https://sibmuseum.ru/news/':'Омский государственный историко-краеведческий музей', 
    
    
}



list_xpaths = [
# '//div[@class="oblique-items"]//div[@class="item"]', 
# '//*[@class="elementor-post__card"]', 
# '//*[@class="reg-news"]//p[@class="news-item"]/following-sibling::*[1]',
# '//*[@class="newsItem__contains"]',
# '//*[@class="field-more-title"]',
# '//*[@class="exhibitions__text"]',
# '//*[@class="news-block exhibition"]',
# '//*[@class="articles-grid"]//*[@class="content"]',
# '//a[@class="element"]',
# '//div[@class="media__inf"]',
# '//a[@class="front_cont h_more_w"]',
# '//div[@class="row row-news-page"]',
# '//div[@class="post"]',
# '//td[@style="padding:3px;"]',
# '//div[@class="news-item"]',
# '//div[@class="site_content_left"]',
# '//section[@class="blog"]//div[@class="col-lg-4"]',
# '//div[@class="novelty clearfix novelty-teaser"]',
# '//div[@class="news__item"]',
# '//li[@class="col-md-4 col-lg-3 col-sm-6 col-xs-12"]',
# '//div[@class="content-box"]',
# '//div[@class="news__post"]',
# '//div[@class="t-feed__col-grid__post-wrapper"]',
# '//a[@class="news-item"]',
# '//div[@class="content-area content-left col-md-8 col-sm-8"]//article',
# '//div[@class="card"]',
# '//h3',
# '//div[@class="single-event single-event-3 mb-30 wow fadeInUp my-single-event-3"]',
# '//div[@class="anonsiblock"]',    
# '//div[@class="text"]',
# '//div[@class="tab-content"]//div[@class="item"]'
'//a[@class="news-card__link"]'
 
    
    
    
              
]




for site, xpath  in zip(d_mus_names.keys(), list_xpaths):
    response = requests.get(site,  verify=False)

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        
        
        target_obj = get_div(tree, xpath)  
        dict_info[site] = target_obj
        
        
    else:
        print(f'Ошибка статуса {response.status_code} для {site}')

In [6]:
df_raw_p = pd.DataFrame([dict_info]).T

In [7]:
df_raw_p = df_raw_p.explode(0)
df_raw_p = df_raw_p.reset_index()

In [8]:
df_raw_p = df_raw_p.rename(columns={'index':'site', 0:'raw_str'})

In [9]:
df_raw_p[df_raw_p['raw_str'].isna()]

,site,raw_str


In [10]:
df_raw_p = df_raw_p[df_raw_p['raw_str'].notna()].copy()

In [11]:
df_raw_p['text_elem'] = df_raw_p['raw_str'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)

In [12]:
df_raw_p = df_raw_p.drop_duplicates(['site', 'text_elem']).copy()
df_raw_p = df_raw_p.reset_index(drop=True).copy()

In [13]:
df_raw_p

,site,raw_str,text_elem
0,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
1,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n выставк...
2,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
3,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n выставк...
4,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n выставк...
5,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
6,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
7,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
8,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
9,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...


## Делаем колоку с хэшом

In [14]:
def generate_hash(input_string):
    # Создаем объект хэша SHA-256
    hash_object = hashlib.sha256()

    # Преобразуем входную строку в байты и обновляем хэш
    hash_object.update(input_string.encode('utf-8'))

    # Получаем шестнадцатеричное представление хэша
    hash_value = hash_object.hexdigest()

    return hash_value

In [15]:
# Колонка для хеширования
df_raw_p.insert(0,'hash_col', df_raw_p['site']+ ' ' + df_raw_p['text_elem'])

df_raw_p['hash_col'] = df_raw_p['hash_col'].apply(generate_hash)

In [16]:
df_raw_p.insert(0,'date_stamp', pd.to_datetime('today'))

In [17]:
df_raw_p

,date_stamp,hash_col,site,raw_str,text_elem
0,2023-12-24 17:42:50.796605,3454949429c6d2666cdba17c59c9a77b6e5629609cb96a...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
1,2023-12-24 17:42:50.796605,7a99be8bea1fc94e93747b6ab98fb8c74560ad3f1131cb...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n выставк...
2,2023-12-24 17:42:50.796605,584ebc08c5057ce6b077904f140611b365f58a778e4677...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
3,2023-12-24 17:42:50.796605,2529d25f35c9ab9e891f4daace599579b5876a3c9c321b...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n выставк...
4,2023-12-24 17:42:50.796605,651b201bf353624a8cc64bfb79570b9ef5f70efeb2276d...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n выставк...
5,2023-12-24 17:42:50.796605,b82d804b50e8c720ac5453123d93cfebb84c40a01393ad...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
6,2023-12-24 17:42:50.796605,3611b4c6501cdb8795f4ab423d76e269a8ca1effac80d1...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
7,2023-12-24 17:42:50.796605,db4f1932edb9df4b6aaadd5874eccca60d9a107bbc963b...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
8,2023-12-24 17:42:50.796605,ce215ebcfe79064ff692991656a2463ff6995f5b89798f...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
9,2023-12-24 17:42:50.796605,261002dc9f857ff2f2efe4a815d76cba0373eb08401074...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...


## Загружаем в db

In [18]:
conn = sqlite3.connect('museums.db')

In [19]:
# Создаю таблицы

qu = '''
    CREATE TABLE if not EXISTS "parse" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT
)
'''
conn.execute(qu)

qu = '''
    CREATE TABLE if not EXISTS "parse_temp" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT

)
'''
conn.execute(qu)

# Добавляю таблицу, для успешных выполнений
qu = '''
    CREATE TABLE if not EXISTS send_telegram (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE, 
    teleg_mes TEXT
)
'''
conn.execute(qu)

In [20]:
df_raw_p

,date_stamp,hash_col,site,raw_str,text_elem
0,2023-12-24 17:42:50.796605,3454949429c6d2666cdba17c59c9a77b6e5629609cb96a...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
1,2023-12-24 17:42:50.796605,7a99be8bea1fc94e93747b6ab98fb8c74560ad3f1131cb...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n выставк...
2,2023-12-24 17:42:50.796605,584ebc08c5057ce6b077904f140611b365f58a778e4677...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
3,2023-12-24 17:42:50.796605,2529d25f35c9ab9e891f4daace599579b5876a3c9c321b...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n выставк...
4,2023-12-24 17:42:50.796605,651b201bf353624a8cc64bfb79570b9ef5f70efeb2276d...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n выставк...
5,2023-12-24 17:42:50.796605,b82d804b50e8c720ac5453123d93cfebb84c40a01393ad...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
6,2023-12-24 17:42:50.796605,3611b4c6501cdb8795f4ab423d76e269a8ca1effac80d1...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
7,2023-12-24 17:42:50.796605,db4f1932edb9df4b6aaadd5874eccca60d9a107bbc963b...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
8,2023-12-24 17:42:50.796605,ce215ebcfe79064ff692991656a2463ff6995f5b89798f...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...
9,2023-12-24 17:42:50.796605,261002dc9f857ff2f2efe4a815d76cba0373eb08401074...,http://ogikm.ru/,"<div class=""item"">\n <d...",\n\n\n событие...


In [21]:
# Временная таблица
df_raw_p.to_sql('parse_temp', conn, index=False, if_exists='replace')


# Импортирую значения с новыми значениями по хэшу
qu = '''
INSERT INTO parse
select *
from parse_temp
where hash_col not in (select hash_col from parse)

'''
conn.execute(qu)

In [22]:
qu = '''
select *
from parse
where hash_col not in (select hash_col from send_telegram)
order by date_stamp
'''
df_n_compl = pd.read_sql(qu, conn)

# Чистка

In [23]:
def adj_lim(x:list):
    '''Ограничивает длину слова'''
    for i, el  in enumerate(x):

        if len(el) > WORLD_LIMIT:
            x[i] = el[:WORLD_LIMIT]+'...'
            
    return x



list_unuse = ['\t\t\t\tЧитать далее»\t\t\t', 'подробнее >>']


def drop_words(x:list):
    '''
    Удаляю лишние слова 
    '''
    new_list = []
    for l in x:

        if not l in list_unuse:
            new_list.append(l)
    return new_list

        

In [24]:
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('^[(\\n)\\t ]+','', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('(\\n){3,}','\n\n', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace(' {3,}',' ', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('\\n+$', '', regex=True)

In [25]:
df_raw_p['text_elem'] = df_raw_p['text_elem'].str.replace('[0-9]* Comments', '')

In [26]:
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(lambda x: re.split(r"(\n)", x))

df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(adj_lim)

df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(drop_words)

In [27]:
df_n_compl['text_elem']= df_n_compl['text_elem'].apply(lambda x: ''.join(x))

In [28]:
df_n_compl

,date_stamp,hash_col,site,raw_str,text_elem
0,2023-12-24 17:42:50.796605,3454949429c6d2666cdba17c59c9a77b6e5629609cb96a...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nПушкинская карта\n\n01 Сентябрь 20...
1,2023-12-24 17:42:50.796605,7a99be8bea1fc94e93747b6ab98fb8c74560ad3f1131cb...,http://ogikm.ru/,"<div class=""item"">\n <d...",выставка \n\nВыставка «На пути сарматского дра...
2,2023-12-24 17:42:50.796605,584ebc08c5057ce6b077904f140611b365f58a778e4677...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nЭкскурсия по выставке «На пути сар...
3,2023-12-24 17:42:50.796605,2529d25f35c9ab9e891f4daace599579b5876a3c9c321b...,http://ogikm.ru/,"<div class=""item"">\n <d...",выставка \n\nПостоянная экспозиция Оренбургско...
4,2023-12-24 17:42:50.796605,651b201bf353624a8cc64bfb79570b9ef5f70efeb2276d...,http://ogikm.ru/,"<div class=""item"">\n <d...",выставка \n\nВыставка «Старинная японская фото...
5,2023-12-24 17:42:50.796605,b82d804b50e8c720ac5453123d93cfebb84c40a01393ad...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nОбзорная экскурсия по основной экс...
6,2023-12-24 17:42:50.796605,3611b4c6501cdb8795f4ab423d76e269a8ca1effac80d1...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nСборная экскурсия по выставке «Ста...
7,2023-12-24 17:42:50.796605,db4f1932edb9df4b6aaadd5874eccca60d9a107bbc963b...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nИнтерактивная экскурсия «Оренбуржц...
8,2023-12-24 17:42:50.796605,ce215ebcfe79064ff692991656a2463ff6995f5b89798f...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nЭкскурсия «Вагон “Аптека-перевязоч...
9,2023-12-24 17:42:50.796605,261002dc9f857ff2f2efe4a815d76cba0373eb08401074...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nМастер-класс по линогравюре (12+)\...


# Telegram

## Добавляем имена музеев

In [29]:
df_n_compl['mus_name'] = df_n_compl['site'].map(d_mus_names)

df_n_compl['mus_name']= df_n_compl['mus_name'].fillna(df_n_compl['site'])


## Отправка

### Подготовка

In [30]:
df_n_compl['teleg_mes'] = '[' + df_n_compl.mus_name + ']' + '(' + df_n_compl.site + ')' + '\n'

df_n_compl['teleg_mes'] = df_n_compl['teleg_mes']+ df_n_compl.text_elem

In [31]:
df_n_compl = df_n_compl[df_n_compl['text_elem']!='']

In [32]:
df_n_compl

,date_stamp,hash_col,site,raw_str,text_elem,mus_name,teleg_mes
0,2023-12-24 17:42:50.796605,3454949429c6d2666cdba17c59c9a77b6e5629609cb96a...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nПушкинская карта\n\n01 Сентябрь 20...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...
1,2023-12-24 17:42:50.796605,7a99be8bea1fc94e93747b6ab98fb8c74560ad3f1131cb...,http://ogikm.ru/,"<div class=""item"">\n <d...",выставка \n\nВыставка «На пути сарматского дра...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...
2,2023-12-24 17:42:50.796605,584ebc08c5057ce6b077904f140611b365f58a778e4677...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nЭкскурсия по выставке «На пути сар...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...
3,2023-12-24 17:42:50.796605,2529d25f35c9ab9e891f4daace599579b5876a3c9c321b...,http://ogikm.ru/,"<div class=""item"">\n <d...",выставка \n\nПостоянная экспозиция Оренбургско...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...
4,2023-12-24 17:42:50.796605,651b201bf353624a8cc64bfb79570b9ef5f70efeb2276d...,http://ogikm.ru/,"<div class=""item"">\n <d...",выставка \n\nВыставка «Старинная японская фото...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...
5,2023-12-24 17:42:50.796605,b82d804b50e8c720ac5453123d93cfebb84c40a01393ad...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nОбзорная экскурсия по основной экс...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...
6,2023-12-24 17:42:50.796605,3611b4c6501cdb8795f4ab423d76e269a8ca1effac80d1...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nСборная экскурсия по выставке «Ста...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...
7,2023-12-24 17:42:50.796605,db4f1932edb9df4b6aaadd5874eccca60d9a107bbc963b...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nИнтерактивная экскурсия «Оренбуржц...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...
8,2023-12-24 17:42:50.796605,ce215ebcfe79064ff692991656a2463ff6995f5b89798f...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nЭкскурсия «Вагон “Аптека-перевязоч...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...
9,2023-12-24 17:42:50.796605,261002dc9f857ff2f2efe4a815d76cba0373eb08401074...,http://ogikm.ru/,"<div class=""item"">\n <d...",событие \n\nМастер-класс по линогравюре (12+)\...,Оренбургский губернаторский историко-краеведче...,[Оренбургский губернаторский историко-краеведч...


In [33]:
df_mes = df_n_compl[['hash_col', 'teleg_mes']].copy()

### Исполнение

In [39]:
df_mes['teleg_mes']

'[Оренбургский губернаторский историко-краеведческий музей ](http://ogikm.ru/)\nвыставка \n\nВыставка «На пути сарматского дракона. Памятники искусства V века до н.э. — III века н.э. из собрания ГМИИ им. А.С. Пушкина» (6+)\n\n27 Октябрь 2023 - 21 Январь 2024\n Выставка, посвящённая важной теме взаимодействия кочевников и огромного мира восточных и античных культур, в кото...Подробнее'

In [40]:
# Убирайю символ не разрывного пробела
df_mes['teleg_mes'] = df_mes['teleg_mes'].str.replace('\xa0', ' ', regex=True)

In [36]:
cnjg

NameError: name 'cnjg' is not defined

In [41]:
execute_session = pd.to_datetime('now')


# Замените 'YOUR_BOT_TOKEN' на токен вашего бота
bot = telebot.TeleBot(BOT_TOKEN)

# Замените 'YOUR_CHAT_ID' на ID чата, куда вы хотите отправить сообщение
chat_id = CHAT_ID

PAUSE_ITER = 20
PAUSE_TIME = 50


# Ожидание перд запуском
# time.sleep(PAUSE_TIME)
for num, tuple_mes in enumerate(df_mes.itertuples()):
    
    try:
        # Отправка сообщения
        bot.send_message(chat_id, tuple_mes.teleg_mes, parse_mode='Markdown')
    
        # Сохранение успешных сообщений
        df = pd.DataFrame([tuple_mes]).drop(columns='Index')

        df['date_stamp'] = execute_session

        if num % 19==0 and num!=0:
            print('Ожидание')
            time.sleep(PAUSE_TIME)

        df.to_sql('send_telegram', conn, if_exists='append', index=False)
    except Exception as send_error:
        print(num, send_error, tuple_mes)
        df_error = pd.DataFrame([[tuple_mes.hash_col, num,send_error, 
                                              tuple_mes.teleg_mes]], columns=['hash_col','num', 'error', 'teleg_mes'])

        df_error.insert(0,'date_time', pd.to_datetime('now'))

        df_error.to_sql('send_error', conn, index=False, if_exists='append')

    
    


Ожидание
